In [18]:
import pandas as pd
import folium

import requests
from urllib.parse import quote

In [56]:
df_park = pd.read_csv('서울시 주요 공원현황(수정).csv')

###  카카오 이용하여 위도 경도 찾기


In [145]:
with open('kakaoapikey.txt', mode='r') as key_file:
    kakao_key = key_file.read()

In [84]:
kakao_key

'a6b6735fc9f59b88acdc81b77a1c1cfb'

In [146]:
len(kakao_key)

32

In [147]:
type(df_park['공원주소'])

pandas.core.series.Series

In [116]:
#공원 주소 가져오기 
addr_list = []
for i in range(len(df_park)):
    addr_list.append(df_park['공원주소'][i]) 
# addr_list

In [117]:
url_list=[]
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
for i in range(len(df_park)):
    url_list.append( f'{local_url}?query={quote(addr_list[i])}')

# url_list

132

In [118]:
# "Authorization: KakaoAK {REST_API_KEY}"
result_list= []
for i in range(len(df_park)):
    result_list.append( requests.get(url_list[i], 
            headers={"Authorization": f"KakaoAK {kakao_key}"}).json()  ) 
# result_list

In [119]:
len(result_list)

132

In [148]:
## 카카오 api를 불러올 떄 에러가 발생함
# 카카오api가 주소를 주지 못하는 에러를 확인 
for i in range(len(df_park)):
    try:
        print(result_list[i]['documents'][0]['x'], result_list[i]['documents'][0]['y'])

    except Exception as e:
        print(e,  df_park['공원명'][i])

126.992217614712 37.5556326132925
127.153328745828 37.5390660422084
127.014098361931 37.4364305503019
127.041799099222 37.5430701468405
126.881888633222 37.5723265509352
126.978413044806 37.5752071090486
126.919058656228 37.637083378098
126.911368630536 37.5197916984599
127.023004390427 37.5575958843951
126.860947770892 37.5574606524359
127.052421806499 37.5448139376614
127.14710389039 37.5503003119836
126.922284053463 37.5268088229325
127.049064320439 37.5727809772819
127.004389310038 37.5669291250632
126.912126417071 37.5161266236525
126.922198893922 37.4907114604033
127.078772391405 37.5520117024951
127.125538378702 37.5439476727563
126.984592154409 37.5201168643347
126.955611774864 37.5746085138333
list index out of range 낙산근린공원
127.036977084936 37.4730498033959
126.959294337648 37.468038057989
126.962290943049 37.544957405495
126.90791394038 37.4696150596404
127.025777287367 37.5972067134377
126.847851799258 37.493179404841
126.97107522045 37.5704968494713
126.967417939446 37.5021

 위도경도 변환 사이트 : https://jusoga.com/b/1111011900100800001000001/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C-%EC%A2%85%EB%A1%9C%EA%B5%AC-%EC%84%B8%EC%A2%85%EB%8C%80%EB%A1%9C-%EC%A7%80%ED%95%98-189-%EC%84%B8%EC%A2%85%EB%A1%9C


### 에러수정  

낙산근린공원 37.580497639748  127.00656325818

자매근린공원 37.517186901579  126.92851352157

와룡근린공원 37.591182301287 126.99011889605

삼청근린공원 37.587690852448 126.98353293438

세종로공원 37.573278995653 126.97627827995



In [121]:
#수정된 에러리스트 # (삭제)
error_list = ['낙산근린공원', '자매근린공원','와룡근린공원', '삼청근린공원', '세종로공원']
error_list_lng = [37.580497639748, 37.580497639748, 37.591182301287, 37.587690852448, 37.573278995653]
error_list_lat = [127.00656325818, 126.92851352157, 126.99011889605, 126.98353293438, 126.97627827995]

In [196]:
# 에러리스트 데이터프레임 만드려고 작업
error_list = ['낙산근린공원', '자매근린공원','와룡근린공원', '삼청근린공원', '세종로공원']
lng_lat= [ ['37.580497639748', '127.00656325818'], ['37.580497639748', '126.92851352157'], ['37.591182301287', '126.99011889605'], ['37.587690852448', '126.98353293438'],['37.573278995653', '126.97627827995'] ]

# 딕셔너리 생성
dic = dict(zip(error_list, lng_lat ) )
dic

{'낙산근린공원': ['37.580497639748', '127.00656325818'],
 '자매근린공원': ['37.580497639748', '126.92851352157'],
 '와룡근린공원': ['37.591182301287', '126.99011889605'],
 '삼청근린공원': ['37.587690852448', '126.98353293438'],
 '세종로공원': ['37.573278995653', '126.97627827995']}

In [206]:
 '낙산근린공원' in dic.keys()

False

In [208]:
# 에러수정 3# 
lng_list, lat_list = [], []
for i in range(len(df_park)):
    try:
        url = f"{local_url}?query={quote(df_park['공원주소'][i])}"
        result = requests.get(url, headers={"Authorization": f"KakaoAK {kakao_key}"}).json()
        print(result_list[i]['documents'][0]['x'], result_list[i]['documents'][0]['y'])
        lng = float(result['documents'][0]['x'])
        lat = float(result['documents'][0]['y'])
    except Exception as e:
        print(e,  df_park['공원명'][i], df_park['공원번호'][i] )
        if df_park['공원명'][i] in dic.keys():
            lng = float(error_list_lng[k])
            lat = float(error_list_lat[k])
    finally:
        lng_list.append(lng)
        lat_list.append(lat)
        print(f" = {df_park['공원번호'][i]}번 공원")      

126.992217614712 37.5556326132925
 = 1번 공원
127.153328745828 37.5390660422084
 = 3번 공원
127.014098361931 37.4364305503019
 = 4번 공원
127.041799099222 37.5430701468405
 = 5번 공원
126.881888633222 37.5723265509352
 = 6번 공원
126.978413044806 37.5752071090486
 = 12번 공원
126.919058656228 37.637083378098
 = 13번 공원
126.911368630536 37.5197916984599
 = 16번 공원
127.023004390427 37.5575958843951
 = 17번 공원
126.860947770892 37.5574606524359
 = 18번 공원
127.052421806499 37.5448139376614
 = 19번 공원
127.14710389039 37.5503003119836
 = 20번 공원
126.922284053463 37.5268088229325
 = 21번 공원
127.049064320439 37.5727809772819
 = 22번 공원
127.004389310038 37.5669291250632
 = 23번 공원
126.912126417071 37.5161266236525
 = 24번 공원
126.922198893922 37.4907114604033
 = 25번 공원
127.078772391405 37.5520117024951
 = 26번 공원
127.125538378702 37.5439476727563
 = 27번 공원
126.984592154409 37.5201168643347
 = 29번 공원
126.955611774864 37.5746085138333
 = 30번 공원
list index out of range 낙산근린공원 31
 = 31번 공원
127.036977084936 37.4730498033959
 = 32

In [209]:
len(lng_list), len(lat_list) # 개수 확인

(132, 132)

In [210]:
df_park['위도'] = lat_list
df_park['경도'] = lng_list
df_park.head(1) # 확인

,공원번호,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,...,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기,위도,경도
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 2454140㎡ 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,...,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107,450395.554,126.990377,37.55014,http://parks.seoul.go.kr/template/sub/namsan.do,37.555633,126.992218


# map 그리기

In [216]:
map = folium.Map(
    location=[df_park['위도'].mean(), df_park['경도'].mean()],  zoom_start=5 )
for i in df_park.index:
    folium.Circle(
    radius=100,
    location=[df_park['위도'][i], df_park['경도'][i]],
    tooltip=df_park['공원명'][i], 
    popup=folium.Popup(
            df_park['면적'][i], min_width=200, max_width=200), 
    color="crimson",
    fill=False,
).add_to(map)

title_html = '<h3 align="center" style="font-size:20px"><b>서울시 공원 데이터 표시</b></h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map